# Übungszettel 2: Python, Korrelation (Musterlösung)

## Maschinelles Lernen - WiSe 23/24

### Abgabe 01.11.2023, 23:55 Uhr

*Hinweise:*
- Übungsaufgaben **müssen** in Gruppen von 3-4 Personen abgegeben werden. 
- Es wird pro Übungszettel nur eine Aufgabe bewertet, die übrigen Aufgaben dienen zur selbstständigen Vertiefung des Vorlesungsstoffs. Für diese Aufgaben werden nach der Abgabe Musterlösungen bereitgestellt.
- Die Lösungen sollen in diesem IPython Notebook realisiert werden, wobei Teilaufgaben und Zwischenergebnisse ausgegeben bzw. visualisiert werden sollen.
- Für die Abgabe sollen Sie dieses IPython Notebook und ggf. zugehörige Dateien in ein **Ziparchiv** packen und im Ilias hochladen. Das Ziparchiv soll nach folgendem Muster benannt werden:
`UebungXX_Nachname1_Nachname2_Nachname3.zip`, wobei die Nachnamen in alphabetischer Reihenfolge angegeben und Umlaute ggf. ersetzt werden sollen.

--- 

## Aufgabe 1: Objektorientierte Programmierung

a) Schreiben Sie eine Klasse `Dice`, die einen Würfel modelliert. Diese soll die Würfe in einer Liste speichern. Die Klasse soll über folgende Funktionen verfügen:

* `roll` - generiert zufällig einen Wert zwischen 1 und 6 und fügt ihn der Historie hinzu
* `history(start, end)` - gibt die Würfe zwischen `start` und `end` aus (für ungültige Eingaben soll eine Exception `InvalidIntervalError` geworfen werden)
* `mean` - gibt den Mittelwert der bereits geworfenen Würfe aus
* `var` - gibt die Varianz der bereits geworfenen Würfe aus

In [13]:
import random

class InvalidIntervalError(Exception):
    pass


class Dice:
    
    def __init__(self):
        self.history_arr = []
        
    def roll(self):
        self.history_arr.append(random.randint(1,6))
            
    def history(self, start, end):
        if 0 <= start <= end <= len(self.history_arr):
            print("history[{}:{}]={}".format(start,end,self.history_arr[start:end]))
        else: 
            raise InvalidIntervalError("interval (start:{}, end:{}) is not valid".format(start,end))
            
    def mean(self, verbose=1):
        m = sum(self.history_arr) / len(self.history_arr)
        if verbose:
            print("mean: {:.2f}".format(m))
        return m
                  
    def var(self, verbose=1):
        m = self.mean(verbose=0)
        v = sum([(x - m)**2 for x in self.history_arr]) / len(self.history_arr)
        if verbose:
            print("variance: {:.2f}".format(v))
        return v 

b) Erstellen Sie drei Mini-Szenarios, in dem eine Würfelinstanz erstellt wird und dann der Würfel {10, 1000, 100000}-mal geworfen wird. Geben Sie anschließend jeweils ein Intervall von 10 Würfen der Historie sowie Mittelwert und Varianz aus.

In [14]:
print("Szenario 1:")    
d1 = Dice()
for _ in range(10):
    d1.roll()

d1.history(0,10)
d1.mean()
d1.var()
    
print("\nSzenario 2:")        
d2 = Dice()
for _ in range(1000): 
    d2.roll()

d2.history(100,110)
d2.mean()
d2.var()
    

print("\nSzenario 3:")    
d3 = Dice()
for _ in range(100000): 
    d3.roll()

d3.history(99990,100000)
d3.mean()
d3.var();   # ; hides the last cell result

Szenario 1:
history[0:10]=[1, 5, 2, 4, 2, 2, 1, 3, 3, 3]
mean: 2.60
variance: 1.44

Szenario 2:
history[100:110]=[4, 4, 2, 1, 5, 4, 4, 3, 1, 2]
mean: 3.52
variance: 2.93

Szenario 3:
history[99990:100000]=[1, 4, 4, 2, 2, 4, 4, 5, 5, 2]
mean: 3.51
variance: 2.93


---

## Aufgabe 2: Webscraping, Dateien ein- und auslesen

Korrelationa) Lesen Sie den Inhalt von https://de.wikipedia.org/wiki/Philipps-Universit%C3%A4t_Marburg ein. Wählen Sie darin alle Bereiche die vom HTML Tag `<div>` mit Attribut `id="mw-content-text"` eingeschlossen werden. Entfernen Sie dann jeweils alle HTML-Tags. Speichern sie den gesamten extrahierten Text in der Datei `Uni_Marburg_Wikipedia.txt`.

*Tipp:* Nutzen Sie die Bibliotheken `beautifulsoup4` und `requests`.

In [11]:
url = 'https://de.wikipedia.org/wiki/Philipps-Universit%C3%A4t_Marburg'
from bs4 import BeautifulSoup
import requests
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

soup = [p.get_text() for p in soup.find_all('div', id='mw-content-text')]
text = "".join(soup)

with open("Uni_Marburg_Wikipedia.txt", "w") as f:
    f.write(text)

b) Lesen Sie die Datei `Uni_Marburg_Wikipedia.txt` ein. Lassen Sie berechnen, wie oft das Wort Informatik vorkommt. Welches sind die 10 Worte, mit mehr als 3 Buchstaben, die im Text am häufigsten vorkommen? Geben Sie diese sortiert zusammen mit ihrer Häufigkeit aus.

In [12]:
with open("Uni_Marburg_Wikipedia.txt", "r") as f:
    text = f.read()
    
print(text.count("Informatik"))
words = text.split()

word_counts = {}

for w in words:
    if len(w) > 3:
        if w not in word_counts.keys():
            word_counts[w] = 1
        else:
            word_counts[w] += 1
             
sorted(word_counts.items(), key=lambda item: item[1], reverse=True)[:10]

1


[('Marburg', 110),
 ('Universität', 58),
 ('Marburger', 55),
 ('Philipps-Universität', 53),
 ('Quelltext', 32),
 ('bearbeiten]', 32),
 ('Hochschule', 30),
 ('eine', 28),
 ('wurde', 27),
 ('Marburg.', 27)]

---
## Aufgabe 3: Pearson-Korrelation

Berechnen Sie die Korrelationskoeffizienten für die folgenden Merkmale:

co2     | year
--------|-----
792.793 | 2014
795.940 | 2015
801.655 | 2016
797.966 | 2017
759.002 | 2018

a) Berechnen Sie die Pearson-Korrelation von Hand und geben Sie den Rechenweg an.

Für die Darstellung des Rechenweges können Sie LaTex in Markdown benutzen. Alternativ können Sie auch ein Foto des Rechenweges einbinden.

$ \text{Hier können Sie Ihre Lösung in } \mathrm{L\!\!^{{}_{A}} \!\!\!\!\!\;\; T\!_{\displaystyle E} \! X} \text{ einfügen...} $

![Bild](https://via.placeholder.com/500x80?text=...oder+ein+Bild+einbinden.)

$$ 
\begin{align}
\bar{x} &= \frac{1}{n} \sum ^n _{i=1} x_i = \frac{1} {5} (792.793 + 795.940 + 801.655 + 797.966 + 759.002) = 789.471 \\
\bar{y} &= \frac{1}{n} \sum ^n _{i=1} y_i = \frac{1} {5} (2014 + 2015 + 2016 + 2017 + 2018) = 2016 \\
\end{align}
$$

$$ 
\begin{align}
s_{xy} &= \frac{1}{n} \sum ^n _{i=1}(x_i - \bar{x})(y_i - \bar{y}) \\
&= \frac{1}{5} ( (792.793 - 789.4712) (2014 - 2016) \\
& \ + (795.940 - 789.471) (2015 - 2016) \\
& \ + (801.655 - 789.471) (2016 - 2016) \\
& \ + (797.966 - 789.471) (2017 - 2016) \\
& \ + (759.002 - 789.471) (2018 - 2016) ) \\
&= \frac{1}{5} ( -6.644 - 6.469 + 0.0 + 8.495 - 60.939) \\
&= -13.111
\end{align}
$$

$$ 
\begin{align}
s_{x} &= \sqrt{ \frac{1}{n} \sum ^n _{i=1}(x_i - \bar{x})^2} \\
&= \sqrt{ \frac{1}{5} ((792.793 - 789.4712)^2 \\
+ (795.940 - 789.471)^2 \\
+ (801.655 - 789.471)^2 \\
+ (797.966 - 789.471)^2 \\
+ (759.002 - 789.471)^2 )} \\
&= \sqrt{ \frac{1}{5} (11.036 + 41.848 + 148.450 + 72.165 + 928.360) } \\
&= \sqrt{ 240.372 } \\
&= 15.504
\end{align}
$$

$$ 
\begin{align}
s_{y} &= \sqrt{ \frac{1}{n} \sum ^n _{i=1}(y_i - \bar{y})^2} \\
&= \sqrt{ \frac{1}{5}  (2014 - 2016)^2 \\
+ (2015 - 2016)^2 \\
+ (2016 - 2016)^2 \\
+ (2017 - 2016)^2 \\
+ (2018 - 2016)^2 )} \\
&= \sqrt{ \frac{1}{5} (4 + 1 + 0 + 1 + 4) } \\
&= \sqrt{ 2 } \\
&= 1.414
\end{align}
$$

$$ 
r_{XY} = \frac{s_{xy}}{s_x * s_y} = \frac{-13.111} {15.504 * 1.414} = -0.598
$$

b) Schreiben Sie eine Funktion welche die Pearson-Korrelation zwischen zwei Merkmalen (zwei Listen von Werten) berechnet. In dieser Teilaufgabe dürfen Sie keine Pakete und insbesondere kein Numpy verwenden.

In [4]:
def mean(x):
    return sum(x)/len(x)

def cov(x,y):
    xm = mean(x)
    ym = mean(y)
    return mean( [(xi-xm)*(yi-ym) for (xi, yi) in zip(x,y)]) 

def std(x):
    xm = mean(x)
    return  mean([(xi-xm)**2 for xi in x])**0.5

def pcc(x,y):
    return cov(x,y)/(std(x) * std(y))

c) Überprüfen Sie Ihr Ergebnis aus Aufgabenteil a) mit Ihrer Implementierung.

In [10]:
co2 = [792.793, 795.940, 801.655, 797.966, 759.002]
year = [2014, 2015, 2016, 2017, 2018]

pcc(co2, year)

-0.5979787889440676

---
## **Aufgabe 4: Spearman-Rangkorrelation (bewertet: 0,5+2,5+1,5+0,5 Punkte)**

Berechnen Sie die Spearman-Rangkorrelationskoeffizienten für die folgenden Merkmale:

co2     | year
--------|-----
792.793 | 2014
795.940 | 2015
801.655 | 2016
797.966 | 2017
759.002 | 2018

a) Stellen Sie eine Tabelle mit den Rängen für die beiden Merkmale auf.

| x=co2   | rx | y=year | ry |
|---------|----|--------|----|
| 792.793 | 2  | 2014   | 1  |
| 795.940 | 3  | 2015   | 2  |
| 801.655 | 5  | 2016   | 3  |
| 797.966 | 4  | 2017   | 4  |
| 759.002 | 1  | 2018   | 5  |

b) Berechnen Sie den Spearman-Rangkorrelationskoeffizienten von Hand und geben Sie den Rechenweg an. Für die Darstellung des Rechenweges können Sie LaTex in Markdown benutzen. Alternativ können Sie auch ein Foto des Rechenweges einbinden.

$$
\begin{align}
\bar{r}_x &= \frac{1}{n} \sum ^n _{i=1} r_{x_i} = \frac{1} {5} (2 + 3 + 5 + 4 + 1) = 3 \\
\bar{r}_y &= \frac{1}{n} \sum ^n _{i=1} r_{y_i} = \frac{1} {5} (1 + 2 + 3 + 4 + 5) = 3 \\
\end{align}
$$

$$ 
\begin{align}
s_{r_xr_y} &= \frac{1}{n} \sum ^n _{i=1}(r_{x_i} - \bar{r}_x)(r_{y_i} - \bar{r}_y) \\
&= \frac{1}{5} ( (2 - 3) (1 - 3) \\
& \ + (3 - 3) (2 - 3) \\
& \ + (5 - 3) (3 - 3) \\
& \ + (4 - 3) (4 - 3) \\
& \ + (1 - 3) (5 - 3) ) \\
&= \frac{1}{5} ( 2 + 0 + 0 + 1 - 4) \\
&= -0.2
\end{align}
$$

$$ 
\begin{align}
s_{r_x} &= \sqrt{ \frac{1}{n} \sum ^n _{i=1}(r_{x_i} - \bar{r}_x)^2} \\
&= \sqrt{ \frac{1}{5} ((2 - 3)^2 \\
+ (3 - 3)^2 \\
+ (5 - 3)^2 \\
+ (4 - 3)^2 \\
+ (1 - 3)^2 )} \\
&= \sqrt{ \frac{1}{5} (1 + 0 + 4 + 1 + 4) } \\
&= \sqrt{ 2 } \\
&= 1.414
\end{align}
$$

$$ 
\begin{align}
s_{r_y} &= \sqrt{ \frac{1}{n} \sum ^n _{i=1}(r_{y_i} - \bar{r}_y)^2} \\
&= \sqrt{ \frac{1}{5}  (1 - 3)^2 \\
+ (2 - 3)^2 \\
+ (3 - 3)^2 \\
+ (4 - 3)^2 \\
+ (5 - 3)^2 )} \\
&= \sqrt{ \frac{1}{5} (4 + 1 + 0 + 1 + 4) } \\
&= \sqrt{ 2 } \\
&= 1.414
\end{align}
$$

$$ 
r_{XY}^{S} = \frac{s_{r_x r_y}}{s_{r_x} * s_{r_y}} = \frac{-0.2} {1.414 * 1.414} = -0.1
$$

c) Schreiben Sie eine Funktion welche den Spearman-Rangkorrelationskoeffizienten zwischen zwei Merkmalen (zwei Listen von Werten) berechnet. In dieser Teilaufgabe dürfen Sie keine Pakete und insbesondere kein Numpy verwenden. 

In [33]:
def mean(x):
    return sum(x)/len(x)

def cov(x,y):
    xm = mean(x)
    ym = mean(y)
    return mean( [(xi-xm)*(yi-ym) for (xi, yi) in zip(x,y)]) 

def std(x):
    xm = mean(x)
    return  mean([(xi-xm)**2 for xi in x])**0.5

def pcc(x,y):
    return cov(x,y)/(std(x) * std(y))


def argsort(x):
    return sorted(range(len(x)), key=x.__getitem__)

def rank(x):
    idx = argsort(x)
    
    # special handling of ranks for same values (not required for this example)
    rank = [0 for _ in x]
    r_val = 0    
    i = j = 0
    while i < len(idx):
        
        if j < len(idx) and x[idx[i]] == x[idx[j]]:  # same value found, average rank
            r_val = (i+j) / 2 + 1
            j += 1
        else:
            rank[idx[i]] = r_val
            i += 1
    
    # no special handling of ranks for same values
    # rank = [i+1 for i in argsort(idx)]
    return rank

def scc(x,y):
    return pcc(rank(x), rank(y))

d) Überprüfen Sie Ihr Ergebnis aus Aufgabenteil b) mit Ihrer Implementierung.

In [35]:
co2 = [792.793, 795.940, 801.655, 797.966, 759.002]
year = [2014, 2015, 2016, 2017, 2018]

scc(co2, year)

-0.09999999999999998